In [1]:
# importing required modules
import idx2numpy
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import torch
import pickle

In [41]:
#importing datasets

# mnist dataset
mnist_train = idx2numpy.convert_from_file("train-images.idx3-ubyte")
mnist_trainlabel = idx2numpy.convert_from_file("train-labels.idx1-ubyte")
mnist_test = idx2numpy.convert_from_file("t10k-images.idx3-ubyte")
mnist_testlabel = idx2numpy.convert_from_file("t10k-labels.idx1-ubyte")
mnist_train = np.expand_dims(mnist_train,-1)
mnist_test = np.expand_dims(mnist_test,-1)



In [9]:
#cifar-10 dataset
def cifar_10_load(file):
    a = list()
    b=  list()
    
    for i in file:
        with open(i, 'rb') as data:
            dict1 = pickle.load(data, encoding='bytes')
            keys = list(dict1.keys())
            b.append(dict1[keys[1]])
            a.append(dict1[keys[2]].reshape(10000,3,32,32).transpose(0,3,2,1))
    
    data = a[0]
    labels = b[0]
    for j in range(1,len(a)):
        data = np.concatenate([data,a[j]])
        labels = np.concatenate([labels,b[j]])
        
    del a,b
    
    return data,labels

train_lists = ["cifar-10-batches-py/data_batch_1",
              "cifar-10-batches-py/data_batch_2",
              "cifar-10-batches-py/data_batch_3","cifar-10-batches-py/data_batch_4",
              "cifar-10-batches-py/data_batch_5"]

with open("cifar-10-batches-py/batches.meta",'rb') as x:
    batch_label = pickle.load(x,encoding='bytes')
    label_name = batch_label[batch_label]
x.close()

cifar_10_train,cifar_10_trainlabels = cifar_10_load(train_lists)
cifar_10_test,cifar_10_testlabels = cifar_10_load(["cifar-10-batches-py/test_batch"])

KeyError: 'label_names'

In [7]:
batch_label

{b'num_cases_per_batch': 10000,
 b'label_names': [b'airplane',
  b'automobile',
  b'bird',
  b'cat',
  b'deer',
  b'dog',
  b'frog',
  b'horse',
  b'ship',
  b'truck'],
 b'num_vis': 3072}

In [36]:
# building softmax_reg_model 

softmax_reg_model = tf.keras.models.Sequential()
softmax_reg_model.add(tf.keras.layers.Flatten(input_shape =mnist_train.shape[1:]))
softmax_reg_model.add(tf.keras.layers.Dense(128,activation='relu'))
softmax_reg_model.add(tf.keras.layers.Dense(128,activation='relu'))
softmax_reg_model.add(tf.keras.layers.Dense(128,activation ='relu'))
softmax_reg_model.add(tf.keras.layers.Dense(10,activation ='softmax'))

softmax_reg_model.compile(optimizer = 'adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics = ['accuracy'])

In [37]:
softmax_reg_model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_13 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_52 (Dense)             (None, 128)               100480    
_________________________________________________________________
dense_53 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_54 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_55 (Dense)             (None, 10)                1290      
Total params: 134,794
Trainable params: 134,794
Non-trainable params: 0
_________________________________________________________________


In [38]:
softmax_reg_model.fit(mnist_train,mnist_trainlabel,validation_split=.2,epochs=5)

Epoch 1/5
1500/1500 [==============================] - 3s 2ms/step - loss: 1.1609 - accuracy: 0.8614 - val_loss: 0.3726 - val_accuracy: 0.9137
Epoch 2/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2540 - accuracy: 0.9319 - val_loss: 0.2363 - val_accuracy: 0.9404
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1907 - accuracy: 0.9474 - val_loss: 0.1720 - val_accuracy: 0.9502
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1561 - accuracy: 0.9550 - val_loss: 0.1817 - val_accuracy: 0.9499
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1361 - accuracy: 0.9609 - val_loss: 0.1532 - val_accuracy: 0.9589


In [ ]:
softmax_reg_model.evaluate(mnist_test,mnist_testlabel)

313/313 [==============================] - 1s 2ms/step - loss: 0.1966 - accuracy: 0.9472


[0.19657449424266815, 0.9472000002861023]

In [9]:
# building Convolution Neural network
cnn_model = tf.keras.Sequential()
cnn_model.add(tf.keras.layers.Conv2D(128,(2,2),input_shape = train.shape[1:],activation='relu',))
cnn_model.add(tf.keras.layers.MaxPool2D((2,2)))
cnn_model.add(tf.keras.layers.Conv2D(128,(2,2),activation='relu'))
cnn_model.add(tf.keras.layers.MaxPool2D((2,2)))
cnn_model.add(tf.keras.layers.Flatten())
cnn_model.add(tf.keras.layers.Dense(128,activation='relu'))
cnn_model.add(tf.keras.layers.Dense(64,activation='relu'))
cnn_model.add(tf.keras.layers.Dense(10,activation='softmax'))

cnn_model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
cnn_model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 27, 27, 128)       640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 12, 12, 128)       65664     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 128)         0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 4608)              0         
_________________________________________________________________
dense_48 (Dense)             (None, 128)               589952    
_________________________________________________________________
dense_49 (Dense)             (None, 64)              

In [10]:
cnn_model.fit(mnist_train,mnist_trainlabel,epochs=5,validation_split=.2)

Epoch 1/5
1500/1500 [==============================] - 5s 4ms/step - loss: 0.2907 - accuracy: 0.9442 - val_loss: 0.0871 - val_accuracy: 0.9725
Epoch 2/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0754 - accuracy: 0.9765 - val_loss: 0.0728 - val_accuracy: 0.9792
Epoch 3/5
1500/1500 [==============================] - 5s 4ms/step - loss: 0.0572 - accuracy: 0.9826 - val_loss: 0.0633 - val_accuracy: 0.9805
Epoch 4/5
1500/1500 [==============================] - 5s 4ms/step - loss: 0.0472 - accuracy: 0.9847 - val_loss: 0.0687 - val_accuracy: 0.9810
Epoch 5/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0382 - accuracy: 0.9885 - val_loss: 0.0772 - val_accuracy: 0.9804


In [ ]:
cnn_model.evaluate(mnist_test,mnist_testlabel)

313/313 [==============================] - 1s 2ms/step - loss: 0.0608 - accuracy: 0.9829


[0.06083374470472336, 0.9829000234603882]

In [12]:
#building muilti layer preceptron model


MLP_model = tf.keras.models.Sequential()
MLP_model.add(tf.keras.layers.Flatten(input_shape = (28,28)))
MLP_model.add(tf.keras.layers.Dense(124,activation='relu'))
MLP_model.add(tf.keras.layers.Dense(124,activation='relu'))
MLP_model.add(tf.keras.layers.Dense(124,activation ='sigmoid'))
MLP_model.add(tf.keras.layers.Dense(10))

MLP_model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

In [13]:
MLP_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 124)               97340     
_________________________________________________________________
dense_15 (Dense)             (None, 124)               15500     
_________________________________________________________________
dense_16 (Dense)             (None, 124)               15500     
_________________________________________________________________
dense_17 (Dense)             (None, 10)                1250      
Total params: 129,590
Trainable params: 129,590
Non-trainable params: 0
_________________________________________________________________


In [15]:
MLP_model.fit(mnist_train,mnist_trainlabel,epochs=5,validation_split=0.2)

NameError: ignored

In [ ]:
MLP_model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_11 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_44 (Dense)             (None, 124)               97340     
_________________________________________________________________
activation_22 (Activation)   (None, 124)               0         
_________________________________________________________________
dense_45 (Dense)             (None, 124)               15500     
_________________________________________________________________
activation_23 (Activation)   (None, 124)               0         
_________________________________________________________________
dense_46 (Dense)             (None, 124)               15500     
_________________________________________________________________
dense_47 (Dense)             (None, 10)              